<a href="https://colab.research.google.com/github/ShadowRaccoon/PrograConcurrente2023C1/blob/main/TP2/TP2_Parte2_M1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Programacion Concurrente TP2 Parte2 MPI

Para este ejercicio se ha optado por aplicar el tema teórico **MPI** (Message Passing Interface). La finalidad del ejercicio es ampliar el conocimiento sobre la manera que posee Python para implementar la comunicación entre distintos procesos con el uso de una librería denominada **MPI4py** [1]. Para más información puede consultar la última revisión en [2].

Este ejercicio fue presentado por alumnos del curso 2020.

---
## 2.1. Ejercicio - Hola Mundo con MPI


### 2.1.1. Preguntas del ejercicio

Ejecute este ejemplo para $4$ o más instancias y responda:

a) ¿Qué función realiza la instancia maestra? ¿Qué función realizan las instancias esclavas?

**Respuesta**

La instancia maestra genera las horas de trabajo y descanso aleatoriamente y envía un set de trabajo/descanso a las instancias esclavas.
Luego de eso, espera a que las instancias esclavas terminen con el set, y cuando alguna de ellas responde, envía un nuevo set a la misma (siempre que quede trabajo pendiente).
Una vez que repartió todo el trabajo, espera a que todas las instancias esclavas terminen con sus sets. Cuando todas las instancias esclavas finalizan todo el trabajo, les envía una bandera de fin de trabajo (END_WORK_FLAG)

Las instancias esclavas, esperan que se les asigne trabajo/descanso. Al recibir trabajo, lo realiza, y al terminar lo notifica a la instancia maestra. Esto se repite hasta que reciben una bandera de fin de trabajo (END_WORK_FLAG).

b) ¿Cuántas de esas instancias ejecuta la función `main()`, `initWork()` y `doWork()`?

**Respuesta**

La función `main()` se ejecuta en una instancia maestra y en N esclavas.
La función `initWork()` se ejecuta en una instancia maestra
La función `doWork()` se ejecuta en N instancias esclavas
Siendo N lo que se haya especificado en el parámetro NRO menos 1.

c) ¿Cómo se diferencian los mensajes de trabajo o de finalización?

**Respuesta**

Los mensajes se diferencian por medio de tags que se envían junto al mensaje. Se usa WORK_FLAG para marcar que el mensaje es de asignación de trabajo y END_WORK_FLAG para marcar la finalización de la jornada de trabajo.

d) ¿Cómo implementaría la función BLAS `axpy()` con este patrón?¿Sería eficiente? *Tips*: Pide solo el planteo, no la implementación.

**Respuesta**

Tendría una instancia maestra que envíe las posiciones con las que tiene que operar.
Las instancias esclavas deberían hacer la operación y comunicar el resultado a la instancia maestra.
Luego de esto, la instancia maestra si tiene más posiciones pendientes, debería enviarselo a la instancia que le haya entregado un resultado, ya que significaría que está libre.
Lo ideal sería tener un número limitado de instancias esclavas que trabajen, y no tantas como posiciones en el vector, ya que esto podría causar mucho overhead y ser contraproducente.
Cuando se termine la operación, se notificaría a las instancias esclavas que no tienen más trabajo que hacer.

e) ¿Qué sucede cuando solo ejecuta con una sola instancia?

**Respuesta**

Si en el parámetro NRO se pone literalmente un 1, no se crean instancias esclavas, sólo se crea la maestra y no se crearía ningun set de trabajo.
  numTasks = (numProcesses-1)*4
Si en el parámetro NRO se pone un 2, tendríamos una instancia maestra y una esclava, por lo que el código se comporta de forma "secuencial" ya que se van sincronizando las instancias.

f) *Punto opcional*: El código que ejecutan las instancias esclavas, tienen un error en su lógica. ¿Cómo se podría solucionar?

**Respuesta**

Las instancias esclavas siempre imprimen por pantalla:
  "Soy el empleado con id X, toca descanzo por 0 hs."
al finalizar todas las tareas pendientes, antes de "marcar tarjeta".
Para solucionar esto, movería el print que muestra ese mensaje debajo de la verficación de la bandera END_WORK_FLAG, para que si ya terminó todo, no muestre ese mensaje.
Además, consideramos que sería buena idea si la bandera no es END_WORK_FLAG, verificar si efectivamente es WORK_FLAG, para evitar cualquier tipo de error si se le estuviera mandando un mensaje incorrecto y permitir escalabilidad de más tipos de mensajes.

Por último, fuera del código de las instancias esclavas, en el método generateTasks(numTasks), está el siguiente comentario no resuelto:

    #TODO: Cambiar la semilla del random para que se generen efectivamente diferentes numeros.

Para solucionar eso, alcanza con quitar la línea en que se setea una semilla, ya que por default, random utiliza la hora del sistema como semilla, que es lo que se suele usar para la generación de números aleatorios.

###2.1.2. Armado del ambiente

Es en donde se instalar, por única vez, el módulo MPI4py de Python en el cuaderno.

In [ ]:
! pip install mpi4py

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 27.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for mpi4py: filename=mpi4py-3.1.4-cp310-cp310-linux_x86_64.whl size=3365684 sha256=9ab7b56678db3e14b094ebb38f385aa92bcaaf095dd7952e87b19a24fdfb54b3
  Stored in directory: /root/.cache/pip/wheels/e8/1b/b5/97ec4cfccdde26e0f3590ad6e09a5242d508dff09704ef86c1
Successfully built mpi4py


### 2.1.3. Código del programa en Lenguaje Python

In [ ]:
%%writefile Ejercicio2.py
from mpi4py import MPI
import numpy as np
import time

# --------------------------------------------
# Formulario
Max_tiempo_sleep =   5#@param {type: "slider", min: 1, max: 10}
Min_tiempo_sleep =   1#@param {type: "slider", min: 0, max: 10}
# --------------------------------------------

# --------------------------------------------
# Constantes de comunicacion
WORK_FLAG = 1
END_WORK_FLAG = 2
# --------------------------------------------

def main():
    comm = MPI.COMM_WORLD # Instanciamos el tipo de comunicador a utilizar.
    id = comm.Get_rank()  # Obtenemos el id como atributo del proceso que se ejecuta.

    # Utilizamos el 0 para definir al procesos Maestro, cualquier otro id sera un esclavo.
    if (id == 0) :
        init() # Llamamos funcion init para eventos que requeriremos inicialmente solo 1 vez.
        numProcesses = comm.Get_size()  # Obtenemos el numero de procesos totales ejecutados.
        numTasks = (numProcesses-1)*4 # Se setea el numero de tareas.
        workTimes = generateTasks(numTasks) # Se generan las tareas, en este caso seran
        print("El jefe crea {} horas de descanso de sus empleados:".format(workTimes.size), flush=True)
        print(workTimes, flush=True)
        initWork(comm, workTimes, numProcesses)
    else:
        doWork(comm)

def generateTasks(numTasks):
    #TODO: Cambiar la semilla del random para que se generen efectivamente diferentes numeros.
    np.random.seed(1000)
    return np.random.randint(low=Min_tiempo_sleep, high=Max_tiempo_sleep, size=numTasks)

def init():
  print()
  print("Version MPI4py utilizada: {}".format(MPI.Get_version()), flush=True)
  print()
  print( "------------------------------------", flush=True)
  print( "Sistema de trabajo Suizo:", flush=True)
  print( "------------------------------------", flush=True)
  print()

def initWork(comm, workTimes, numProcesses):
    totalWork = workTimes.size
    workcount = 0
    recvcount = 0

    print("Jefe enviando las tareas iniciales:", flush=True)
    for id in range(1, numProcesses):
        if workcount < totalWork:
            work=workTimes[workcount]
            comm.send(work, dest=id, tag=WORK_FLAG) # Envia mensaje de iniciar trabajo con el dato correspondiente del array.
            workcount += 1
            print("Jefe envia trabajo y {} hs de descanso al empleado {}.".format(work, id), flush=True)
    print( "------------------------------------", flush=True)

    # Mientras haya trabajo, se recibe el resultado de los empleados y se sigue enviando MAS trabajo.
    while (workcount < totalWork) :
        stat = MPI.Status()
        workTime = comm.recv(source=MPI.ANY_SOURCE, status=stat) # Recivimos resultados de los empleados.
        recvcount += 1
        workerId = stat.Get_source() # Obtenemos el identificador del empleado.
        print("Jefe recibe trabajo completado {} del empleado {}.".format(workTime, workerId), flush=True)
        #send next work
        work=workTimes[workcount]
        comm.send(work, dest=workerId, tag=WORK_FLAG)
        workcount += 1
        print("Jefe envia nuevo trabajo y {} hs de descanso al empleado {}.".format(work, workerId), flush=True)

    while (recvcount < totalWork):
        stat = MPI.Status()
        workTime = comm.recv(source=MPI.ANY_SOURCE, status=stat)
        recvcount += 1
        workerId = stat.Get_source()
        print("Jefe recibe trabajo completado {} del empleado {}.".format(workTime, workerId), flush=True)

    for id in range(1, numProcesses):
        comm.send(0, dest=id, tag=END_WORK_FLAG)


def doWork(comm):
    while(True):
        stat = MPI.Status() # Obtiene el estado actual del empleado.
        waitTime = comm.recv(source=0, tag=MPI.ANY_TAG, status=stat) # Obtiene lo enviado por el Jefe.
        print("Soy el empleado con id {}, toca descanzo por {} hs.".format(comm.Get_rank(), waitTime), flush=True)

        if (stat.Get_tag() == END_WORK_FLAG):
            print("Marca tarjeta el empleado {}.".format(comm.Get_rank()), flush=True)
            return
        time.sleep(waitTime)
        comm.send(waitTime, dest=0)

main()


Overwriting Ejercicio2.py


### 2.1.4 Ejecución del programa

In [ ]:
# --------------------------------------------
#@title Parámetros de ejecución { vertical-output: true }
NRO =   5#@param {type: "number"}
# --------------------------------------------

! mpirun --allow-run-as-root --oversubscribe -np $NRO python Ejercicio2.py


Version MPI4py utilizada: (3, 1)

------------------------------------
Sistema de trabajo Suizo:
------------------------------------

El jefe crea 16 horas de descanso de sus empleados:
[4 4 4 1 4 3 1 2 2 1 2 2 1 2 1 4]
Jefe enviando las tareas iniciales:
Jefe envia trabajo y 4 hs de descanso al empleado 1.
Jefe envia trabajo y 4 hs de descanso al empleado 2.
Jefe envia trabajo y 4 hs de descanso al empleado 3.
Jefe envia trabajo y 1 hs de descanso al empleado 4.
Soy el empleado con id 3, toca descanzo por 4 hs.
Soy el empleado con id 4, toca descanzo por 1 hs.
------------------------------------
Soy el empleado con id 1, toca descanzo por 4 hs.
Soy el empleado con id 2, toca descanzo por 4 hs.
Jefe recibe trabajo completado 1 del empleado 4.
Soy el empleado con id 4, toca descanzo por 4 hs.
Jefe envia nuevo trabajo y 4 hs de descanso al empleado 4.
Jefe recibe trabajo completado 4 del empleado 3.
Jefe envia nuevo trabajo y 3 hs de descanso al empleado 3.
Soy el empleado con id 3, t

---
# Bibliografía

[1] MPI4py: https://mpi4py.readthedocs.io/en/stable/

[2] La versión oficial de MPI (Versión 4): https://www.mpi-forum.org/docs/mpi-4.0/mpi40-report.pdf

